# Effects of Drag Resistance
## Lecture 4

Going forward with the course, we will start every Jupyter notebook by importating commonly used libraries.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive

In the last two lectures we have been discussing how to solve problems numerically and how to write programs. 

Here is prototypical simluation program that we have been using to model a falling particle:

In [ ]:
def FallingParticleApp():
    
    # set up the model
    model = FallingParticleStepper()
    
    # iterate the model
    ball = pd.DataFrame(model)
    
    # plot the results
    FallingParticlePlot(ball)

We separate out the code that will be iterated to step through time to actual solve the numerical program:

In [ ]:
def FallingParticleStepper(dt=0.01, y0=100, v0=0, tmax=5):
    
    # initialize the model
    g = 9.8
    t = 0
    y = y0
    v = v0
    
    while True:
        
        model = {'t': t, 'y': y, 'v': v,
                 'v0': v0, 'y0': y0, 'dt': dt, 
                }
        yield model # return to the caller
            
        if t > tmax:
            break
            
        # use the Euler algorithm to update the state of the model
        y = y + v*dt
        v = v - g*dt
        t = t + dt

And the routines to visualize and plot the results:

In [ ]:
def FallingParticlePlot(particle):
    
    plt.plot(particle.t, particle.y)
    plt.ylabel('y (m)')
    plt.xlabel('t (s)')

In [ ]:
FallingParticleApp()

## Review of a Falling Particle

The above code simulates a projectile on the surface of the Earth with no air friction. 

Newton's Second Law gives use the equation of motion, $F = m a = m \frac{\mathrm{d}^2y}{\mathrm{d}t^2} $ with $F = - m g$ for which we can rewrite as the pair of equations

$$\begin{align}
\frac{\mathrm{d}y}{\mathrm{d}t} &= v \\
\frac{\mathrm{d}v}{\mathrm{d}t} &= -g. \\
\end{align}
$$

We then approximated the derivatives with finite differences

$$\begin{align}
\frac{y(t+\Delta t) - y(t)}{\Delta t} &= v(t) \\
\frac{v(t+\Delta t) - v(t)}{\Delta t} &= -g 
\end{align}
$$

or rearranged gives

$$
\begin{align}
y(t + \Delta t) & = y(t) + v(t) \Delta t \\
v(t + \Delta t) & = v(t) -g \Delta t.
\end{align}
$$

This algorithm tells how how to estimate new values for the position $y$ and velocity $v$ from the values at the previous time step. In code, this is written as

```
y = y + v*dt
v = v - g*dt
```

The analytical soluiton for free fall near the Earth's surface,

$$y(t) = y_0 + v_0  t - \frac{1}{2}g t^2,$$

is well known and thus finding a numerical solution is useful only as an introduction to numerical programming. 

The more general problem is to have an arbitrary sum of forces $F_{\mathrm{net}}$ acting on a particle. Then the equation of motion $F_{\mathrm{net}} = m  \frac{\mathrm{d}^2y}{\mathrm{d}t^2} $ leads to 

$$\begin{align}
\frac{\mathrm{d}y}{\mathrm{d}t} &= v \\
\frac{\mathrm{d}v}{\mathrm{d}t} &= \frac{F_{\mathrm{net}}}{m} . \\
\end{align}
$$

Approximating the derivatives as finite differences leads again to Euler's method:

$$
\begin{align}
y(t + \Delta t) & = y(t) + v(t) \Delta t \\
v(t + \Delta t) & = v(t) + \frac{F_{\mathrm{net}}}{m}  \Delta t.
\end{align}
$$

or 
```
y = y + v*dt
v = v + Fnet/m*dt
```

It is important to recognize that that Euler's method is a numerical algorithm for solving differential equations. There is nothing special at all about the problem being about a falling particle.  In the most general case, we can approximately solve the  differential equation

$$ \frac{\mathrm{d}y}{\mathrm{d}t} = f(y, t)$$

with the algorithm

$$y(t + \Delta t) = y(t) + f(y(t), t) \Delta t$$

where $f(y, t)$ is any mathematical function of $y$ and (optionally) $t$.

It is not difficult to think of more realistic models of motion near the Earth's surface for which the equations of motion do not have simple analytical solutions. For example, if we take into account the variation of the Earth's gravitational field with the distance from the centre of the Earth, then the force on a particle is not constant. According to Newton's law of gravitation, the force is due to the Earth on a particle of mass $m$ is given by

$$ F = \frac{GMm}{(R+y)^2} = \frac{GMm}{R^2(1+y/R)^2} = mg (1 - 2 \frac{y}{R} + \ldots),$$

where $y$ is measured from the Earth's surface, $R$ is the radius of the Earth, $M$ is the mass of the Earth, $G$ is the gravitaional constant, and $g = GM/R^2$.

- - -

### **Problem 3.7** Position-dependent force

Extend `FallingParticleStepper()` and `FallingParticleApp()` to simulate the fall of a particle with the position-dependent force law $F = mg\left(1-2\frac{y}{R}\right)$. Assume that a particle is dropped from a height $h$ with zero initial velocity and compute its impact velocity (speed) when it hits the ground at $y= 0$. Determine the value of $h$ for which the impact velocity differs by one percent from its value with a constant acceleration $g=9.8\; \mathrm{m/s}^2$. Take $R= 6.37\times10^6\;\mathrm{m}$. Make sure that the one percent difference is due to the physics of the force law and not the accuracy of your algorithm (this can be confirmed by showing that the result changes by signifcantly less than one percent if you half the size of your time step).

*(will be part of Assignment 2)*
- - - 

## Drag force due to air resistance

For particles near the Earth's surface, a more important modication is to include the drag force due to air resistance.  

The direction of the drag force $F_{d}$ is opposite to the velocity of the particle. 

For a falling body, $F_{d}$ is upward since the velocity is downwards. 

Hence the total force $F$ on a falling body is

$$ F = - m g + F_{d}.$$



### Empirical model for drag force

Two common ways of modelling a drag force are

$$ F_{d}(v) =  C_1 v, $$

and
$$ F_{d}(v) = C_2 v^2,$$

where $C_1$ and $C_2$ depend on the properties of the fluid and the shape of the object.

Unlike kinetic friction between two sliding surfaces which depends only on the normal force, drag friction depends on the speed, $v$,  of the object.

At very low velocities, the behaviour is essentially linear (this is called **Stokes's Law**) and is represented by the $C_1$ term the linear dependence on $v$.

At more reasonable velocities the behaviour is quadratic as given by the $C_2$ term.

From fluid dynamics, exact values of $C_2$ can be computed only for idealized shapes such as spheres and cylinders. For real objects with more complicated geometries $C_2$ can not be calculated and must be measured from experiment.  We can, however, rewrite the quadratic the drag force as

 $$ F_d =  \frac{1}{2} C_d \rho A v^2$$

where $\rho$ is the density of air, $A$ is the frontal cross-sectional area of the falling object, and $C_d$ is called the **drag coefficient**.  $C_d$ depends on the aerodynamics and shape of an object and it is typically between 0.5 and 1.5 in value.


### Explanation of mathematical form of drag law

We can understand why the air friction has this particular mathematical form by considering the block of air that is being pushed out of the way by a falling object.  The mass of air moved in time $dt$ is
$$m_{\mbox{air}} \sim \rho A v dt. $$

The air is given a velocity of order $v$, and hence a kinetic energy of
$$E_\mbox{air} \sim \frac{1}{2} m_\mbox{air} v^2 .$$

Because of conservation of energy, this is also the work done by the drag force (the force of the object due to air resistance, work is force times displacement) in time $dt$, 
$$ F_{\mbox{d}} v dt = E_\mbox{air}.$$

Putting these three equations together we find

$$F_\mbox{d} \propto \frac{1}{2} \rho A v^2.$$

The constant of proportionality, $C_d$, is determined by experiment.

Notice that we are using the notation 

$$ F_d = C_2 v^2 = \frac{1}{2} C_d \rho A v^2$$

so this really means $C_2 = \frac{1}{2} C_d \rho A$. Both forms of a quadratic drag law are used and, somewhat confusingly, both $C_2$ and $C_d$ are often called a *drag coefficient*.  

## Terminal Velocity

Because $F_d(v)$ increases as $v$ increases, there is a limiting or *terminal velocity* (speed) at which the net force on a falling object is zero.

From 
$$ F = - m g + F_d$$
we can determine the terminal velocity by setting $F_d = mg$ to find

$$\begin{align}
v_{1,t} &= \frac{mg}{C_1} , & \mbox{(linear drag)} \\
v_{2,t} &= \left(\frac{mg}{C_2}\right)^{1/2}, & \mbox{(quadratic drag)} 
\end{align}
$$
for the linear and quadratic cases, respectively.

It is often to express velocities in terms of the terminal velocity. We can use the drag forces and the terminal speeds to write $F_d$ in the linear and quadratic cases as

$$\begin{align}
F_{1,d} &= C_1 v_{1,t}\left(\frac{v}{v_{1,t}}\right) = mg \frac{v}{v_{1,t}}, \\
F_{2,d} &= C_2 v_{2,t}^2 \left(\frac{v}{v_{2,t}}\right)^2 = mg \left(\frac{v}{v_{2,t}}\right)^2.
\end{align}
$$

So, we can write the net force (per unit mass) on a falling object in the convenient forms

$$
\begin{align}
F_1(v)/m &= -g \left( 1 - \frac{v}{v_{1,t}} \right) \\
F_2(v)/m &= -g \left( 1 - \frac{v^2}{v_{2,t}^2} \right)
\end{align}
$$

## Falling pebble

Suppose a pebble of mass $m = 10^{-2}\;\mbox{kg}$ is thrown over the side of high cliff of height $h = 50\;\mbox{m}$. To a good approximation the drag force is proportional to $v^2$.  

For spherical pebble of radius $0.01\;\mbox{m}$, $C_2$ is found empirically to be approximately $10^{-4}\;\mbox{kg/m}$.  This means that the terminal speed is about 30 m/s.

In [ ]:
C2 = 1e-4
m = 1e-2
g = 9.81
v2t = (m*g/C2)**(1/2)
print("v2t = {:.1f}".format(v2t))

Since this speed would be achieved by a freely falling body in a vertical fall of approximately 50 m in a time of about 3 s, we expect that the effects of air resistance would be appreciable for comparable times and distances.

- - -

### **Problem 3.9** Effect of air resistance on the ascent and descent of a pebble

a. Compute the speed at which a pebble reaches the ground if it is dropped from rest at a height of $50\;\mbox{m}$. Compare this speed to that of a freely falling object under the same conditions. Assume that the drag force is proportional to $v^2$ and that the terminal velocity is 30 m/s.

b. Suppose a pebble is thrown vertically upward with an initial velocity $v_0$. In the absence of air resistance, we know that the maximum height reached by the pebble is $ v_0^2/2g$, its velocity upon return to the Earth equals $v_0$, the time of ascent equals the time of descent, and the total time in the air is $2 v_0/g$. Before doing a simulation, give a simple qualitative explanation of how you think these quantities will be affected by air resistance. In particular, how will the time of ascent compare with the time of descent?

c. Do a simulation to determine if your qualitative answers in part (b) are correct. Assume that the drag force is proportional to $v^2$. Choose the coordinate system with $y$ positive upward. What is the net force for $v >0$ and $v <0$? We can characterize the magnitude of the drag force by a terminal velocity even if the motion of the pebble is upward and even if the pebble never attains this velocity. Choose the terminal velocity $v_t= 30\;\mbox{m/s}$, corresponding to a drag coefficient of $C_2 \approx 0.000109$. It is a good idea to choose an initial velocity that allows the pebble to remain in the air for a time sufficiently long so that the effect of the drag force isappreciable. A reasonable choice is $v(t= 0) = 50 \mbox{m/s}$. You might find it convenient to express the drag force in the form $F_d\propto - \mathrm{v * abs(v)}$. 

*(will be part of Assignment 2)*
- - -

## Falling speed as function of cross-sectional area

We saw that we could also write the quadratic expression for the drag law as

$$F_d = \frac{1}{2} C_d \rho A v^2$$

This model suggests the imporance of minimizing the frontal cross-sectional area $A$ in order to achieve a higher maximum speed. 

Let's explore this model to try work out how the falling speed depends on the cross-sectional area of a falling object.

From Newton's law, the equation of motion is

$$\begin{align}
\frac{\mathrm{d}y}{\mathrm{d}t} &= v \\
\frac{\mathrm{d}v}{\mathrm{d}t} &= \frac{F_{\mathrm{net}}}{m} . \\
\end{align}
$$

where

$$\begin{align}
Fnet &= F_g + F_d \\
     &= \left(-mg\right) + \left(\frac{1}{2} C_d \rho A v^2 \right) 
\end{align}
$$

In code, this means this would be our Euler's method:

```
    y = y + v*dt
    v = v + Fnet/m*dt
```

where

`Fnet = Fg + Fd`.

Calculating the force force of gravity is straightforward:

`Fg = - m * g`.

And calculating the drag force involves the following expression:

`Fd = 0.5 * Cd * ρ * A * v**2`


Python supports not only the 26 upper and lower case letters of the Latin alphabet, but many other character sets too. In fact, Python identifiers can be composed of many other Unicode characters.

For example, the symbol ρ (U+03C1, Greek Small Letter Rho) is commonly used to represent density. So we can use it directly in our code: 

In [ ]:
# International Standard Atmosphere states the density of air at sea level and 15 degrees C
ρ = 1.225 # kg/m3 

Depending on your keyboard or operating system you may have a way to type these characters. On an ASCII keyboard, Jupyter allows you to enter these kind of math symbols using

`\rho<TAB>`

That, the characters `\rho` immediately followed by the Tab key. 

In [ ]:
ρ

To be specific, let's model a falling paper lunch bag and investigate its speed depending on its cross sectional area.

<img src="https://images.uline.com/is/image/content/dam/images/S/S7500/S-7084.jpg?$MediumRHD$&iccEmbed=1&icc=AdobeRGB" alt="Paper Lunch Bag" width="200"> 

https://www.uline.ca/Product/Detail/S-7084/Retail-Bags-Paper/Paper-Grocery-Bags-6-1-8-x-4-x-12-3-8-8-Kraft


500 bags in a bundle weigh 6 kg. So, let's estimate the mass of each bag as

In [ ]:
m = 6 / 500
print(m)

For a completely opened paper bag, the dimensions are

In [ ]:
width = 0.16 # m
depth = 0.10 # m
height = 0.31 # m

The cross-sectional area is

In [ ]:
A = width * depth
A

assuming we are dropping the bag with the bottom oriented downward. In this simulation we will vary this value but 0.016 m$^2$ is a good starting point.

Finally, these paper bags are rectangular prisms and have drag coefficient on the order of $C_d \approx 1.0$.

We first write code to solve the problem numerically:

In [ ]:
def PaperBagStepper(A=0.016,
                      dt=0.01, y0=10, v0=0, tmax=2):
    """
    Model a falling object of cross-sectional area A.
    """
        
    # initialize the model
    g = 9.8 # m/s2
    ρ = 1.225 # kg/m3, air
    Cd = 1.0 
    m = 0.012 # kg
    
    t = 0
    y = y0
    v = v0
    
    while True:
        model = {'t': t, 'y': y, 'v': v}
        yield model # return to the caller
            
        if t > tmax:
            break
        
        Fg = -m*g
        Fd = 0.5 * Cd * ρ * A * v**2
        Fnet = Fg + Fd
        
        # use the Euler algorithm to update the state of the model
        y = y + v*dt
        v = v + Fnet/m*dt
        t = t + dt
    
    # return the final state of the model
    model = {'t': t, 'y': y, 'v': v}
    yield model

We need at least one function to visualize the results. Here we are using the function `plt.subplots()` to create two axes. In the top one we plot the position vs time. and the bottom one we plot the velocity vs time.

In [ ]:
def FirstPaperBagPlot(bag):
    fig, axs = plt.subplots(2, 1)
    axs[0].plot(bag.t, bag.y)
    axs[0].set_ylabel('y (m)')
    
    axs[1].plot(bag.t, bag.v)
    axs[1].set_ylabel('v (m)')
    axs[1].set_xlabel('t (s)')

And a driver program to put it all together

In [ ]:
def FirstPaperBagApp(A=0.016):
    
    model = PaperBagStepper(A=A)
    bag = pd.DataFrame(model)
    
    FirstPaperBagPlot(bag)

In [ ]:
FirstPaperBagApp()

We can quickly make this application iteractive to explore the dependence of the trajectory on the cross-sectional area.

In [ ]:
interactive(FirstPaperBagApp, A=(0.001, 0.050, 0.001))

To extract out the terminal speed we need to look at the last row in the data frame.

In [ ]:
model = PaperBagStepper()
bag = pd.DataFrame(model)
bag.tail()

In Pandas, you can reference a particular row with `.iloc[]`

In [ ]:
bag.iloc[201]

Using the index `-1` gives you the last row in the data frame

In [ ]:
bag.iloc[-1]

We may want to show the the results for many different cross-sectional areas all in one plot.

In [ ]:
def SecondPaperBagApp():
    
    # set of cross-sectional areas
    As = np.arange(0.005, 0.05, 0.005)
    
    # list to hold the terminal speeds, v
    vs = []
    
    # we want to reuse the figure axes so we create a single set of axes
    # here in the driver program
    fig, axs = plt.subplots(2, 1)
    for A in As:
        model = PaperBagStepper(A=A, dt=0.001)
        bag = pd.DataFrame(model)
    
        # take the absolute value of velocity as the terminal speed
        v = abs(bag.iloc[-1].v)
        # store this value for later
        vs.append(v)
        
        # the axes are passed into the plotting function
        SecondPaperBagPlot(bag, axs)
        
    plt.subplots(1, 1)
    plt.plot(As, vs, 'o-')
    plt.xlabel('Cross-sectional area $A$ (m$^2$)')
    plt.ylabel('Terminal Speed $v$ (m/s)')
    

In [ ]:
def SecondPaperBagPlot(bag, axs):
    axs[0].plot(bag.t, bag.y)
    axs[0].set_ylabel('y (m)')
    
    axs[1].plot(bag.t, bag.v)
    axs[1].set_ylabel('v (m)')
    axs[1].set_xlabel('t (s)')

In [ ]:
SecondPaperBagApp()

This shows, as expected, as the cross-sectional area increases the terminal speed decreases.

## Falling speed as a function of humidity

From 

$$F_d = \frac{1}{2} C_d \rho A v^2$$

we can also see that the drag force depends on the air density. While the the density of air is around $\rho = 1.3$ kg/m$^3$, the precise density depends on the pressure, temperature, and humidity of the air.

Based on some [formulae found on the web](http://physics.holsoft.nl/physics/ocmain.htm#densair) (unfortuantely, the link is now broken), here is a function that determines the density of air as function of relative humidity. 

In [ ]:
def calc_air_density(RH=0, T=20, B=101325):
    """
    Calculate the density of humid air given
        relative humidiy, RH (0.00 - 1.00)
        temperature, T (Celsius)
        barometric pressure, B (Pa)
    """
    # temperature in Kelvin
    T_kelvin = T + 273.15
    # saturated vapour pressure
    p_sat = 610.7 * np.power(10, 7.5*T/(237.3+T))
    # partial vapour pressure
    p_v = RH * p_sat
    # apply formula for density of humid air
    rho_air = 1.2929 * 273.15/T_kelvin * (B - 0.3783 * p_v) / 1.013e5
    
    return rho_air

Relative humidity is a percentage between 0% and 100% (0.00 to 1.00) indicating the water content of the air for a given temperature and pressure.

- - -
### *Exercise*

Explore how relatively humidity affects the terminal speed of a falling object.

> 1. Modify the code below to investigate the impact of relative humidity on the maximum velocity.
> 2. Use the function `calc_air_density()` to compute the air density as needed.
> 3. Considering only the impact of humidity on the drag force, what does the model predict about the change in the terminal speed?
> 4. How does your model compare with your intuition on the impact of humidity on terminal speed? 

*(in-class exercise)*

In [ ]:
def FallingHumidityStepper(RH=0.0,
                           dt=0.01, y0=10, v0=0, tmax=2):
    """
    Model a falling object of in a relative humidity RH.
    """
        
    # initialize the model
    g = 9.8 # m/s2
    A = 0.02 # m2
    Cd = 1.0 
    m = 0.012 # kg
    
    # Density is no longer a constant
    #ρ = 1.225 # kg/m3, air
    
    t = 0
    y = y0
    v = v0
    
    while True:
        model = {'t': t, 'y': y, 'v': v}
        yield model # return to the caller
            
        if t > tmax:
            break

        # fill in the code to determine the 
        # density from the relative humidity
        ρ = ___
        
        Fg = -m*g
        Fd = 0.5 * Cd * ρ * A * v**2
        Fnet = Fg + Fd
        
        # use the Euler algorithm to update the state of the model
        y = y + v*dt
        v = v + Fnet/m*dt
        t = t + dt
    
    # return the final state of the model
    model = {'t': t, 'y': y, 'v': v}
    yield model

In [ ]:
def FallingHumidityPlot(bag, axs):
    axs[0].plot(bag.t, bag.y)
    axs[0].set_ylabel('y (m)')
    
    axs[1].plot(bag.t, bag.v)
    axs[1].set_ylabel('v (m)')
    axs[1].set_xlabel('t (s)')

In [ ]:
def FallingHumidityApp():
    
    # set of relative humidities
    RHs = np.arange(___, ___, ___)
    
    # list to hold the terminal speeds, v
    vs = []
    
    # we want to reuse the figure axes so we create a single set of axes
    # here in the driver program
    fig, axs = plt.subplots(2, 1)
    for ___ in ___:
        model = FallingHumidityStepper(___)
        bag = pd.DataFrame(model)
    
        # take the last velocity as the terminal speed
        v = bag.iloc[-1].v
        # store this value for later
        vs.append(v)
        
        # the axes are passed into the plotting function
        FallingHumidityPlot(bag, axs)
        
    plt.subplots(1, 1)
    plt.plot(RHs, vs, 'o-')
    plt.xlabel('Relative Humidity $RH$ (%)')
    plt.ylabel('Terminal Speed $v$ (m/s)')

In [ ]:
FallingHumidityApp()

Discuss what we find as a class.

- - -
Textbook readings

This lecture is based on CSM Chapter 3:

- 3.7 Effect of Drag Resistance
